<a href="https://colab.research.google.com/github/dsenanayake95/violence_detection/blob/master/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import os, sys
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [ ]:
# tensorflow imports
from tensorflow.keras import Sequential, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model

In [ ]:
!git clone https://github.com/dsenanayake95/violence_detection.git

fatal: destination path 'violence_detection' already exists and is not an empty directory.


In [ ]:
%cd violence_detection/

/content/violence_detection


In [ ]:
CWD_PATH = sys.path.append(os.getcwd())

In [ ]:
cd ..

/content


In [ ]:
def preprocess(image, label):
    image = image / 255
    return image, label


def augment(image, label):
    if np.random.rand(1) < 0.2:
        image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.4)
    return image, label

In [ ]:
# ZIP = "raw_data/violence.zip"  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHANGE
# CWD_PATH = os.getcwd()
ROOT = "drive/MyDrive/violence_detection/raw_data/cropped_dataset"
# BUCKET_NAME = "YOURBUCKETNAME"  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHANGE

# TARGET_IMSIZE = (224,224)

AUTOTUNE = tf.data.experimental.AUTOTUNE

# Build a trainer class
class Trainer():
    def __init__(self, root):
        self.train_generator = None
        self.test_generator = None
        self.val_generator = None
        self.model = None
        self.history = None
        self.eval_ = None
        self.root = ROOT # data root path

    # def unzipper(self):
    #     with zipfile.ZipFile(ZIP, 'r') as zip_ref:
    #         zip_ref.extractall(".")
    #     return self

    # Splitting frames into train, test
    def split(self):
        self.train_ratio = 0.8
        self.test_ratio = 0.2

        classes_dir = ['violence', 'non_violence'] # all labels

        for label in classes_dir:
            os.makedirs(ROOT + '/train/' + label)
            os.makedirs(ROOT + '/test/' + label)

            # Creating partitions of the data after shuffling
            src = ROOT + "/" + label  # Folder to copy images from

            self.allFileNames = os.listdir(src)
            np.random.shuffle(self.allFileNames)
            self.train_FileNames, self.test_FileNames = np.split(np.array(self.allFileNames),
                                                                  [int(len(self.allFileNames)* (1 - (self.test_ratio)))])
                                                                  

            self.train_FileNames = [src+'/'+ name for name in self.train_FileNames.tolist()]
            self.test_FileNames = [src+'/' + name for name in self.test_FileNames.tolist()]

            # Copy-pasting images
            for name in self.train_FileNames:
                shutil.copy(name, ROOT + '/train/' + label)

            for name in self.test_FileNames:
                shutil.copy(name, ROOT + '/test/' + label)


    # Generate data + Augment frames in the train set
    def generate_data(self):
        train_dir = ROOT + "/train/"

        self.train_generator = image_dataset_from_directory(train_dir,
                                                    labels='inferred',
                                                    seed=123,
                                                    label_mode='binary',
                                                    batch_size=16,
                                                    image_size=(224,224),
                                                    shuffle=True,
                                                    subset='training',
                                                    validation_split=0.33)
        
        self.val_generator = image_dataset_from_directory(train_dir,
                                                    labels='inferred',
                                                    seed=123,
                                                    label_mode='binary',
                                                    batch_size=16,
                                                    image_size=(224,224),
                                                    shuffle=True,
                                                    subset='validation',
                                                    validation_split=0.33)

        test_dir = ROOT + "/test/"

        self.test_generator = image_dataset_from_directory(test_dir,
                                                    labels='inferred',
                                                    seed=123,
                                                    label_mode='binary',
                                                    batch_size=16,
                                                    image_size=(224,224),
                                                    shuffle=True)
        
        return self.train_generator, self.val_generator, self.test_generator

    # Transfer learning model
    def load_model(self, transfer=VGG19, dense_n=512, lr= 0.000134):
        transfer_model = transfer(include_top=False, 
                                  weights="imagenet",
                                  input_shape=(224, 224, 3), 
                                  pooling="max", 
                                  classes=2,
                                  )
        
        transfer_model.trainable = False
        
        self.model = tf.keras.models.Sequential([
                                transfer_model,
                                Flatten(),
                                Dense(dense_n, activation='relu'),
                                Dense(dense_n, activation='relu'),
                                Dense(dense_n/2, activation='relu'),
                                Dense(dense_n/4, activation='relu'),
                                Dense(dense_n/8, activation='relu'),
                                Dense(dense_n/16, activation='relu'),
                                Dense(dense_n/32, activation='relu'),
                                Dense(1, activation='sigmoid')
                                    ])

        opt = optimizers.Adam(learning_rate=lr)
        self.model.compile(loss='binary_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
        
        return self.model

    # Instantiate  + fit model
    def run(self, epochs=5):

        train = self.train_generator
        val = self.val_generator
        test = self.test_generator

        train_prep = train.map(augment, num_parallel_calls=AUTOTUNE).map(preprocess, num_parallel_calls=AUTOTUNE)
        val_prep = val.map(preprocess, num_parallel_calls=AUTOTUNE)
        test_prep = test.map(preprocess, num_parallel_calls=AUTOTUNE)

        train_final = train_prep.cache()
        train_final = train_prep.prefetch(AUTOTUNE)

        val_final = val_prep.prefetch(AUTOTUNE) # check if cache could be useful
        test_final = test_prep.prefetch(AUTOTUNE)

        model = self.load_model()

        es = EarlyStopping(monitor='val_accuracy',
                           mode='max',
                           patience=20,
                           verbose=1,
                           restore_best_weights=True)


        model.fit(train_final,
                  validation_data=val_final,
                  epochs=epochs,
                  verbose=1,
                  callbacks=es)
        
        model.evaluate(test_final)

        self.model = model

        return self

    # save the model
    def save(self, name):
        save_model(self.model, f'drive/MyDrive/violence_detection/models/{name}')


In [ ]:
print("loading trainer...")
trainer = Trainer(root=ROOT)
print("trainer loaded")

loading trainer...
trainer loaded


In [ ]:
# print("spliting the data into train, test...")
# trainer.split()
# print("data successfully split")
# print('Total images: ', len(trainer.allFileNames))
# print('Training: ', len(trainer.train_FileNames))
# print('Testing: ', len(trainer.test_FileNames))

In [ ]:
print("augmenting data now...")
trainer.generate_data()
print("data successfully prepped")
print("loading models...")

models = [MobileNet, ResNet50, VGG19]
epochs = [10, 20, 50]
lrs = [0.000134, 0.0013, 0.0112]

optimal_model = VGG19
optimal_lr = 0.0002
epoch = 50

print(f"using learning rate of {optimal_lr} and {optimal_model}")
trainer.load_model(transfer=optimal_model, dense_n=512, lr=optimal_lr)
print(f"{optimal_model} successfully loaded")
print("running model...")
trainer.run(epochs=epoch)
print("model completed")



augmenting data now...
Found 9047 files belonging to 2 classes.
Using 6062 files for training.
Found 9047 files belonging to 2 classes.
Using 2985 files for validation.
Found 2262 files belonging to 2 classes.
data successfully prepped
loading models...
using learning rate of 0.0002 and <function VGG19 at 0x7f096d8ab830>
<function VGG19 at 0x7f096d8ab830> successfully loaded
running model...
Epoch 1/50
379/379 [==============================] - 110s 267ms/step - loss: 0.6877 - accuracy: 0.5501 - val_loss: 0.6761 - val_accuracy: 0.5836
Epoch 2/50
379/379 [==============================] - 93s 244ms/step - loss: 0.6641 - accuracy: 0.5996 - val_loss: 0.6102 - val_accuracy: 0.6740
Epoch 3/50
379/379 [==============================] - 93s 244ms/step - loss: 0.6383 - accuracy: 0.6333 - val_loss: 0.7118 - val_accuracy: 0.6452
Epoch 4/50
379/379 [==============================] - 93s 244ms/step - loss: 0.6157 - accuracy: 0.6554 - val_loss: 0.6575 - val_accuracy: 0.6680
Epoch 5/50
379/379 [====

In [ ]:
trainer.save('VGG19_lr_0.0002_model_v3')

INFO:tensorflow:Assets written to: drive/MyDrive/violence_detection/models/VGG19_lr_0.0002_model_v3/assets


In [ ]:
# testing learning rate of 0.000134
# Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_1_0_224_tf_no_top.h5
# 17227776/17225924 [==============================] - 0s 0us/step
# 17235968/17225924 [==============================] - 0s 0us/step
# <function MobileNet at 0x7fa2c9483f80> successfully loaded
# running model...
# Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5
# 80142336/80134624 [==============================] - 1s 0us/step
# 80150528/80134624 [==============================] - 1s 0us/step
# Epoch 1/5
# 371/371 [==============================] - 1193s 3s/step - loss: 0.6551 - accuracy: 0.5858 - val_loss: 0.6938 - val_accuracy: 0.6459
# Epoch 2/5
# 371/371 [==============================] - 85s 227ms/step - loss: 0.5587 - accuracy: 0.6993 - val_loss: 0.6648 - val_accuracy: 0.6866
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5437 - accuracy: 0.7145 - val_loss: 0.6574 - val_accuracy: 0.7068
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4986 - accuracy: 0.7392 - val_loss: 1.0704 - val_accuracy: 0.6579
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4922 - accuracy: 0.7502 - val_loss: 0.6337 - val_accuracy: 0.7079
# 139/139 [==============================] - 290s 2s/step - loss: 0.6657 - accuracy: 0.7073
# model completed
# testing learning rate of 0.0013
# <function MobileNet at 0x7fa2c9483f80> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 86s 228ms/step - loss: 0.6590 - accuracy: 0.5937 - val_loss: 0.7864 - val_accuracy: 0.6010
# Epoch 2/5
# 371/371 [==============================] - 85s 229ms/step - loss: 0.5893 - accuracy: 0.6773 - val_loss: 0.8023 - val_accuracy: 0.6637
# Epoch 3/5
# 371/371 [==============================] - 85s 227ms/step - loss: 0.5560 - accuracy: 0.7035 - val_loss: 0.6319 - val_accuracy: 0.7106
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5160 - accuracy: 0.7409 - val_loss: 0.6737 - val_accuracy: 0.7103
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4972 - accuracy: 0.7499 - val_loss: 0.6114 - val_accuracy: 0.7373
# 139/139 [==============================] - 22s 152ms/step - loss: 0.6356 - accuracy: 0.7290
# model completed
# testing learning rate of 0.0112
# <function MobileNet at 0x7fa2c9483f80> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 87s 229ms/step - loss: 0.6518 - accuracy: 0.6067 - val_loss: 0.7612 - val_accuracy: 0.6161
# Epoch 2/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5639 - accuracy: 0.6952 - val_loss: 0.8945 - val_accuracy: 0.6651
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5299 - accuracy: 0.7226 - val_loss: 0.9526 - val_accuracy: 0.6527
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5151 - accuracy: 0.7325 - val_loss: 0.6300 - val_accuracy: 0.6914
# Epoch 5/5
# 371/371 [==============================] - 85s 227ms/step - loss: 0.4996 - accuracy: 0.7423 - val_loss: 0.8421 - val_accuracy: 0.6774
# 139/139 [==============================] - 22s 152ms/step - loss: 0.8496 - accuracy: 0.6852
# model completed
# testing learning rate of 0.000134
# Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# 94773248/94765736 [==============================] - 1s 0us/step
# 94781440/94765736 [==============================] - 1s 0us/step
# <function ResNet50 at 0x7fa2c94233b0> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 86s 229ms/step - loss: 0.6285 - accuracy: 0.6176 - val_loss: 0.6695 - val_accuracy: 0.6664
# Epoch 2/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5531 - accuracy: 0.7053 - val_loss: 0.8439 - val_accuracy: 0.6647
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5339 - accuracy: 0.7205 - val_loss: 0.6758 - val_accuracy: 0.7086
# Epoch 4/5
# 371/371 [==============================] - 85s 227ms/step - loss: 0.5014 - accuracy: 0.7443 - val_loss: 0.8785 - val_accuracy: 0.6842
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4881 - accuracy: 0.7482 - val_loss: 0.8376 - val_accuracy: 0.6955
# 139/139 [==============================] - 22s 153ms/step - loss: 0.8548 - accuracy: 0.6920
# model completed
# testing learning rate of 0.0013
# <function ResNet50 at 0x7fa2c94233b0> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 87s 229ms/step - loss: 0.6474 - accuracy: 0.6117 - val_loss: 0.7140 - val_accuracy: 0.6534
# Epoch 2/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5739 - accuracy: 0.6880 - val_loss: 0.6225 - val_accuracy: 0.7140
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5362 - accuracy: 0.7160 - val_loss: 1.1103 - val_accuracy: 0.5925
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5183 - accuracy: 0.7333 - val_loss: 0.6688 - val_accuracy: 0.7048
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4946 - accuracy: 0.7532 - val_loss: 0.7494 - val_accuracy: 0.7041
# 139/139 [==============================] - 22s 152ms/step - loss: 0.7481 - accuracy: 0.7096
# model completed
# testing learning rate of 0.0112
# <function ResNet50 at 0x7fa2c94233b0> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 87s 229ms/step - loss: 0.6353 - accuracy: 0.6154 - val_loss: 0.6828 - val_accuracy: 0.6236
# Epoch 2/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5626 - accuracy: 0.7035 - val_loss: 0.8722 - val_accuracy: 0.6373
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5252 - accuracy: 0.7274 - val_loss: 0.7368 - val_accuracy: 0.6949
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5172 - accuracy: 0.7340 - val_loss: 0.7165 - val_accuracy: 0.6962
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4931 - accuracy: 0.7514 - val_loss: 0.6676 - val_accuracy: 0.6983
# 139/139 [==============================] - 22s 152ms/step - loss: 0.6766 - accuracy: 0.6915
# model completed
# testing learning rate of 0.000134
# <function VGG19 at 0x7fa2c942a830> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 87s 229ms/step - loss: 0.6412 - accuracy: 0.6072 - val_loss: 1.0407 - val_accuracy: 0.5818
# Epoch 2/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5618 - accuracy: 0.6902 - val_loss: 0.5933 - val_accuracy: 0.6904
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5220 - accuracy: 0.7268 - val_loss: 1.0402 - val_accuracy: 0.6616
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5116 - accuracy: 0.7419 - val_loss: 0.8672 - val_accuracy: 0.6514
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4948 - accuracy: 0.7565 - val_loss: 0.6020 - val_accuracy: 0.7390
# 139/139 [==============================] - 22s 153ms/step - loss: 0.6151 - accuracy: 0.7258
# model completed
# testing learning rate of 0.0013
# <function VGG19 at 0x7fa2c942a830> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 87s 229ms/step - loss: 0.6341 - accuracy: 0.6262 - val_loss: 0.6483 - val_accuracy: 0.6870
# Epoch 2/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5606 - accuracy: 0.6969 - val_loss: 0.6535 - val_accuracy: 0.6613
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5435 - accuracy: 0.7138 - val_loss: 0.6393 - val_accuracy: 0.7202
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5091 - accuracy: 0.7419 - val_loss: 0.7926 - val_accuracy: 0.6918
# Epoch 5/5
# 371/371 [==============================] - 85s 229ms/step - loss: 0.5014 - accuracy: 0.7489 - val_loss: 0.5873 - val_accuracy: 0.7175
# 139/139 [==============================] - 22s 153ms/step - loss: 0.6038 - accuracy: 0.7069
# model completed
# testing learning rate of 0.0112
# <function VGG19 at 0x7fa2c942a830> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 87s 230ms/step - loss: 0.6340 - accuracy: 0.6153 - val_loss: 0.7726 - val_accuracy: 0.6445
# Epoch 2/5
# 371/371 [==============================] - 85s 229ms/step - loss: 0.5534 - accuracy: 0.7082 - val_loss: 0.6633 - val_accuracy: 0.6853
# Epoch 3/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5185 - accuracy: 0.7333 - val_loss: 0.6877 - val_accuracy: 0.7110
# Epoch 4/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.5031 - accuracy: 0.7495 - val_loss: 0.6473 - val_accuracy: 0.6925
# Epoch 5/5
# 371/371 [==============================] - 85s 228ms/step - loss: 0.4970 - accuracy: 0.7497 - val_loss: 0.7174 - val_accuracy: 0.7164
# 139/139 [==============================] - 22s 153ms/step - loss: 0.7383 - accuracy: 0.7096

In [ ]:
# Results of different learning rates [0.000134, 0.0013, 0.0112]

# testing learning rate of 0.000134
# <function MobileNet at 0x7f043ca1f0e0> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 91s 241ms/step - loss: 0.4837 - accuracy: 0.7607 - val_loss: 0.4096 - val_accuracy: 0.8123
# Epoch 2/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.3201 - accuracy: 0.8642 - val_loss: 0.3310 - val_accuracy: 0.8493
# Epoch 3/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.2593 - accuracy: 0.8966 - val_loss: 0.2573 - val_accuracy: 0.8932
# Epoch 4/5
# 371/371 [==============================] - 89s 240ms/step - loss: 0.2088 - accuracy: 0.9165 - val_loss: 0.2640 - val_accuracy: 0.8990
# Epoch 5/5
# 371/371 [==============================] - 89s 240ms/step - loss: 0.1601 - accuracy: 0.9368 - val_loss: 0.2874 - val_accuracy: 0.8976
# 139/139 [==============================] - 23s 162ms/step - loss: 0.2827 - accuracy: 0.8916
# model completed
# testing learning rate of 0.0013
# <function MobileNet at 0x7f043ca1f0e0> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 92s 242ms/step - loss: 0.4560 - accuracy: 0.7910 - val_loss: 0.3296 - val_accuracy: 0.8596
# Epoch 2/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.3060 - accuracy: 0.8722 - val_loss: 0.3121 - val_accuracy: 0.8736
# Epoch 3/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.2587 - accuracy: 0.8921 - val_loss: 0.2595 - val_accuracy: 0.8921
# Epoch 4/5
# 371/371 [==============================] - 90s 240ms/step - loss: 0.2211 - accuracy: 0.9123 - val_loss: 0.2750 - val_accuracy: 0.8911
# Epoch 5/5
# 371/371 [==============================] - 90s 240ms/step - loss: 0.1768 - accuracy: 0.9300 - val_loss: 0.3069 - val_accuracy: 0.8805
# 139/139 [==============================] - 23s 161ms/step - loss: 0.2776 - accuracy: 0.8803
# model completed
# testing learning rate of 0.0112
# <function MobileNet at 0x7f043ca1f0e0> successfully loaded
# running model...
# Epoch 1/5
# 371/371 [==============================] - 91s 241ms/step - loss: 0.4707 - accuracy: 0.7792 - val_loss: 0.4468 - val_accuracy: 0.7925
# Epoch 2/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.3214 - accuracy: 0.8578 - val_loss: 0.3648 - val_accuracy: 0.8411
# Epoch 3/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.2648 - accuracy: 0.8883 - val_loss: 0.2665 - val_accuracy: 0.8887
# Epoch 4/5
# 371/371 [==============================] - 89s 240ms/step - loss: 0.2112 - accuracy: 0.9133 - val_loss: 0.2556 - val_accuracy: 0.9038
# Epoch 5/5
# 371/371 [==============================] - 90s 241ms/step - loss: 0.1758 - accuracy: 0.9249 - val_loss: 0.2367 - val_accuracy: 0.9103
# 139/139 [==============================] - 23s 160ms/step - loss: 0.2171 - accuracy: 0.9115